In [2]:
%pip install llama-index-embeddings-openai
%pip install llama-index-vector-stores-pinecone
%pip install llama-index-llms-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# pip install llama-index
# pip -q install python-dotenv pinecone-client llama-index pymupdf

# Setup

In [12]:
import pinecone
import os
from dotenv import load_dotenv
import fitz
from llama_index.core.node_parser import SentenceSplitter


In [ ]:
load_dotenv(dotenv_path='secrets.env')

In [7]:
api_key = os.environ['PINECONE_API_KEY']
environment = os.environ['PINECONE_ENVIRONMENT']
# print(api_key)
pinecone.init(api_key=api_key, environment=environment)
index_name = "llamaindex-rag-fs"

e3c97720-762d-4950-8921-ea13e710368e


In [ ]:
pinecone.delete_index(index_name)
# delete before re-running

In [ ]:
pinecone.create_index(
    index_name, dimension=1536, metric="euclidean", pod_type="p1"
) # text-embedding-ada-002
pinecone_index = pinecone.Index(index_name)

## Create PineconeVectorStore

In [ ]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

## Load Data

In [ ]:
file_path = "./data/llama2.pdf"
doc = fitz.open(file_path)
# print(doc.get_page_text(0))

## Text splitter

In [ ]:
text_parser = SentenceSplitter(
    chunk_size = 1024
)

In [ ]:
text_chunks = []
doc_idxs = []
for doc_idx, page in enumerate(doc):
    page_text = page.get_text("text")
    cur_text_chunks = text_parser.split_text(page_text)
    text_chunks.extend(cur_text_chunks)               # textfrompg1 textfrompg1 textfrompg1 textfrompg2 textfrompg2
    doc_idxs.extend([doc_idx] * len(cur_text_chunks)) # 1 1 1 2 2

# Construct Nodes from Text chunks
Convert chunks into `TextNode` objects.